In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from pandas import Series, DataFrame
from selenium.webdriver.common.keys import Keys
import time
import re

In [3]:
#키를 파일 이름으로 저장
def data_to_csv(dict_n):
    '''
    입력 : 딕셔너리 데이터
    출력 : csv파일 형태로 원시 자료 폴더에 출력.
    '''
    git_address = 'C:/Users/taehee/Documents/GitHub'
    for key, an_attracrion_dict in dict_n.items():
        an_attracrion_dict.to_csv(git_address + '/tripReviewAnalysisSystem/크롤러-전처리/원시자료/' + key + '.csv', mode = 'w')

## HTML을 개발자 도구로 뜯어보자

적절한 선택자를 찾아보자

### 즐길거리 상세 보기
attractions-attraction-overview-pois-PoiInfo__info--239IR 라는 class를 가진 div
안에
h3 

단, 그냥 h3로 찾으면 위에 있는 투어나 다른 추천도 모두 찾게 되므로 유의하자.

### 리스트 더 보기 see more
see more은
attractions-attraction-overview-main-TopPOIs__see_more--2Vsb- 라는 class를 가진 div
로 찾을 수 있다

In [4]:
def reviews(sample_size):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 표본 리뷰의 숫자자.
    맥락 : 리뷰를 추출할 즐길거리 페이지에 있다.
    출력 : 한 즐길거리에 대한 리뷰를 모아놓은 데이터 프레임

    각 리뷰에서 새로운 항목을 추출하고 싶다면, columns와 for문을 모두 수정해야한다.
    '''
    reviews = []
    columns = ['star_point','title','text','Date of experience']
    #,'reviewer_address'
        
        
    #리뷰 크기를 텍스트로 찾아와, 콤마를 제거하여, 정수형으로 저장.
    review_size_text = driver.find_element_by_class_name('attractions-community-content-TabBarContent__tabCount--2hTdj').text
    review_size = int(re.sub(',','', str(review_size_text)))

    n = 5
    count = 0
    while n < sample_size:
        try:
            foot = driver.find_element_by_class_name('location-review-review-list-parts-ReviewScrollController__container--3XtvE')
            show_more = foot.find_element_by_tag_name('button')  
            driver.execute_script("arguments[0].click();", show_more)
            n += 5
        except:
            if count > 3:
                break
            count +=1
            time.sleep(2)
                
    review_cards = driver.find_elements_by_class_name('location-review-card-Card__ui_card--2Mri0')

    for review_card in review_cards:
        try:
            read_more = review_card.find_element_by_class_name('location-review-review-list-parts-ExpandableReview__cta--2mR2g')
            read_more.click()
        except:
            print('error')
            continue
    
    # 드라이버 
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    review_elements = soup.find_all(class_ = 'location-review-card-Card__ui_card--2Mri0 location-review-card-Card__card--o3LVm location-review-card-Card__section--NiAcw')
    
    for review_element in review_elements:
        try:
            review = [
                int(review_element.find('span',class_='ui_bubble_rating')['class'][1][-2:]) / 10,
                review_element.find('a', class_='location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT').find('span').find('span').text,
                review_element.find('q', class_='location-review-review-list-parts-ExpandableReview__reviewText--gOmRC').find('span').text,
                review_element.find('span', class_='location-review-review-list-parts-EventDate__event_date--1epHa').contents[1]
                #,review_element.find('span', class_='social-member-common-MemberHometown__hometown--3kM9S').contents[1]
            ]
        except IndexError:
            #아무도 helpful이나 vote를 누르지 않은 에러, 무시하고 계속
            continue
        except AttributeError:
            #주소를 공개하지 않는 에러, 무시하고 계속
            continue
        reviews.append(review)

    return DataFrame(reviews, columns = columns)

In [5]:
html = "<q class=\"location-review-review-list-parts-ExpandableReview__reviewText--gOmRC\"><span>If you have little to no knowledge of Korean history (like me), I'd suggest you to take the guided tour. The guide was friendly, knowledgeable and very accommodating. <br><br>Book your ticket to Secret Garden online and you can book 7 days earlier prior to your visit. I took the last tour of the day and there were less people in the evening.</span></q>"
soup = BeautifulSoup(html, 'html.parser')
text = soup.find('span').text
print(text)

If you have little to no knowledge of Korean history (like me), I'd suggest you to take the guided tour. The guide was friendly, knowledgeable and very accommodating. Book your ticket to Secret Garden online and you can book 7 days earlier prior to your visit. I took the last tour of the day and there were less people in the evening.


In [6]:
#driver.switch_to.window(original_window)
#driver.current_window_handle;

In [7]:
def load_reviews(link):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 어떤 즐길거리 번호 int, 이름 string 예: "Gyeongbokgung Palace"
    출력 : 어트랙션의 리뷰 모음. data.frame
    '''
    #새 창을 열고 링크로 간다.
    # 슬립...매우매우중요이거없으면무조건에러남.이거하나로3시간머리싸맸음.
    
    
    time.sleep(5)
    original_window = driver.current_window_handle
    driver.execute_script("arguments[0].click();", link)
    
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break
    
    #즐길거리 별로 샘플사이즈를 어떻게 정할 것인지?    
    result = reviews(300)
    
    if result['text'].isnull().sum() != 0:
        print(result['text'].isnull().sum())
        result.head()

    #창을 닫고 리스트 창으로 돌아간다
    driver.close();
    driver.switch_to.window(original_window)
    driver.current_window_handle;
    
    return result
    #result.head()

In [8]:
def load_page(start, end):
    '''
    단위 테스트 성공. 개발 완료.
    입력 : 크롤링할 페이지 내 시작 번호와 끝 번호
    출력 : 페이지의 리뷰들을 csv파일로 출력
    '''

    #모든 여행지의 리스트를 만든다
    is_first = True
    print("I am in the load_page")
    if driver.current_url == 'https://www.tripadvisor.com/Attractions-g294197-Activities-Seoul.html':
        main_page = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__container--3eHZU')
        attraction_elements = main_page.find_elements_by_class_name("attractions-attraction-overview-pois-PoiInfo__info--239IR")
        is_first = True
    else:
        main_page = driver.find_element_by_class_name('scrollAdMain')
        attraction_elements = main_page.find_elements_by_class_name('listing_info')
        is_first = False
    print("I found the elements!")

    attraction_names = []
    for attraction_element in attraction_elements[start:end]:      
        if is_first:
            link = attraction_element.find_element_by_tag_name('h3')
            attraction_name = link.text[link.text.find('.')+2 :]
        else:
            div_class = attraction_element.find_element_by_class_name('tracking_attraction_title')
            link = div_class.find_element_by_tag_name('a')
            attraction_name = link.text
        attraction_names.append((attraction_name, link))   
    print("I found the attraction_names")
        
    attraction_dict = {}
    for attraction_name, link in attraction_names:
        if len(attraction_dict) < end:
            stime = time.time()
            attraction_dict[attraction_name] = load_reviews(link)
            print(attraction_name,  " " + str(time.time() -stime) + "초 걸림 ", len(attraction_dict[attraction_name]))
            if len(attraction_dict) % 5 == 0:
                data_to_csv(attraction_dict)
                print("저장했어요!")
        else:
            data_to_csv(attraction_dict)
            print("저장했어요!")
            break

### 다음페이지 버튼 찾기
attractions-attraction-overview-main-Pagination__container--PUXGq 리스트를 먼저 찾고

현재 페이지는 
attractions-attraction-overview-main-Pagination__selected--2updu 라는 class를 가진 div
로 찾을 수 있다

현재 페이지 + 1 해서 다음 페이지로 가는 a href를 찾는다

In [13]:
def dict_n(start, end):
    '''
    입력 : 크롤링할 즐길거리의 시작 번호와 끝 번호
    출력 : 해당 구간의 즐길거리 리뷰들csv로 출력
    '''    
    #처음 한 번만 see more을 누른다.
    is_first_page = True
    try:
        time.sleep(1)
        show = driver.find_element_by_class_name('attractions-attraction-overview-main-TopPOIs__see_more--2Vsb-')
        driver.execute_script("arguments[0].click();", show)
        is_first_page = True
        print("it is first page!")
    except NoSuchElementException:
        print('continue!')
        is_first_page = False
    except:
        print("exception")
    
    time.sleep(5)
    keep_going = True
    while keep_going:
        
        # 현재 페이지 번호 알아내기

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        print("I made a soup")
        if is_first_page:
            selected_page = 1
            page_numbers = driver.find_element_by_class_name('attractions-attraction-overview-main-Pagination__container--PUXGq')
            if start // 30 > 0:
                if start // 30 > 6:
                    target_num = 6
                else:
                    target_num = (start // 30) + 1
                target_page = page_numbers.find_element_by_link_text(str(target_num))
                time.sleep(5)
                driver.execute_script("arguments[0].click();", target_page)
                time.sleep(3)
                selected_page = target_num
                page_numbers = driver.find_element_by_class_name('pageNumbers')
                is_first_page = False
        else:
            selected_page = int(soup.find("div", class_='pageNumbers').find(class_ = 'current').text)
            page_numbers = driver.find_element_by_class_name('pageNumbers')
        print("I find the current page")
        
        # 시작점이 뒷 페이지면 지나간다다
        if start <= selected_page * 30:
            strat_in_page = start-(selected_page-1)*30
            if strat_in_page < 0:
                strat_in_page = 0
            assert strat_in_page <= 30
            
            end_in_page = end-(selected_page-1)*30
            if end_in_page > 30:
                end_in_page = 30                
            assert 0 <= end_in_page
            load_page(strat_in_page, end_in_page)
            
            if is_first_page:
                is_first_page = False
        
        # 끝나는 지점을 지나갔으면 루프를 끝낸다.
        if (selected_page - 1) * 30 >= end:
            keep_going = False
            break
        # 다음 페이지로 넘어가기
        time.sleep(5)
        next_num = str(selected_page + 1)
        next_page = page_numbers.find_element_by_link_text(next_num)
        driver.execute_script("arguments[0].click();", next_page)


In [15]:
# 언어를 영어 캐나다다로 설정. 이래야 show more버튼이 나오는 줄 알았는데...
options = webdriver.ChromeOptions()
options.add_argument("lang=en-CA")
#이미지를 보지 않음
prefs = {"profile.managed_default_content_settings.images": 2}
options.add_experimental_option("prefs", prefs)

driver_address = 'C:/Users/taehee/Downloads/chromedriver_win32/chromedriver'

is_show_more = False
while not is_show_more:
    driver = webdriver.Chrome(driver_address, chrome_options=options)
    wait = WebDriverWait(driver, timeout=2)
    driver.implicitly_wait(5)
    # tripadvisor 경복궁에 접근한다.
    driver.get('https://www.tripadvisor.com/Attraction_Review-g294197-d324888-Reviews-Gyeongbokgung_Palace-Seoul.html')
    time.sleep(1)
    try:
        foot = driver.find_element_by_class_name('location-review-review-list-parts-ReviewScrollController__container--3XtvE')
        show_more = foot.find_element_by_tag_name('button')
    except:
        driver.close();
        time.sleep(1)
        continue
    is_show_more =True

#원래 있던 리스트 창을 저장해둔다
time.sleep(10)
driver.get('https://www.tripadvisor.com/Attractions-g294197-Activities-Seoul.html')

dict_n(90,180)

#dict_n() => load_page() => load_reviews() => reviews()

C:\Users\taehee\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: use options instead of chrome_options
  if sys.path[0] == '':


it is first page!
I made a soup
I find the current page
I am in the load_page
I found the elements!
I found the attraction_names
error
Seoul Botanical Garden  45.154632568359375초 걸림  7
error
error
error
Mangwon Market  52.86116099357605초 걸림  12
error
National Museum of Korean Contemporary History  69.52338576316833초 걸림  63
error
Express Bus Terminal Shopping Center  57.223050117492676초 걸림  43
error
Kyunghee University Seoul Campus  49.98764991760254초 걸림  19
저장했어요!
error
Common Ground  87.85551881790161초 걸림  98
error
Yongma Land  46.069666385650635초 걸림  11
error
IFC Mall  64.06931209564209초 걸림  60
error
SRT  47.75826168060303초 걸림  17
error
Statue of Admiral Yi Sun-Sin  91.24737787246704초 걸림  100
저장했어요!
error
Korea Furniture Museum  50.76321840286255초 걸림  27
error
Lotte World Folk Museum  51.884936571121216초 걸림  27
error
Lotte Outlets Seoul Station  70.46464943885803초 걸림  71
error
Garak Market  48.47062039375305초 걸림  17
error
Seoul Daehangno  51.38227105140686초 걸림  21
저장했어요!
error
Grille